In [1]:
import asyncio
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient
from helpers import get_current_date
import os
from azure.identity.aio import (ClientSecretCredential, DefaultAzureCredential,
                                get_bearer_token_provider)
from autogen_agentchat.agents import UserProxyAgent
from fmputils import *
from yfutils import *
from autogen_core.tools import FunctionTool
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from coding import *
from text import *
from datetime import date, timedelta, datetime
from reports import *
from charting import *
from textwrap import dedent
from analyzer import *
import tempfile
from helpers import summarize, summarizeTopic

In [2]:
from dotenv import load_dotenv
load_dotenv(".env", override=True)

True

In [3]:
work_dir = "report"
os.makedirs(work_dir, exist_ok=True)

In [4]:
model_client = AzureOpenAIChatCompletionClient(
    azure_deployment=os.getenv('AZURE_OPENAI_DEPLOYMENT_NAME'),
    azure_endpoint=os.getenv('AZURE_OPENAI_ENDPOINT'),
    model="gpt-4o",
    api_version=os.getenv('AZURE_OPENAI_API_VERSION'),
    # azure_ad_token_provider=GetTokenProvider(
    #                 "https://cognitiveservices.azure.com/.default"
    #             ),
    api_key=os.getenv('AZURE_OPENAI_KEY'),
    model_capabilities={
        "vision": False,
        "function_calling": True,
        "json_output": True,
    }
)

In [5]:
def analyze_company_description(ticker_symbol:str, year:str) -> str:
    companyDesc = ReportAnalysisUtils.analyze_company_description(ticker_symbol, year)
    summarized = summarize(companyDesc)
    return (
        f"##### Company Description\n"
        f"**Company Name:** {ticker_symbol}\n"
        f"**Company Analysis:** {summarized}\n"
        #f"{formatting_instructions}"
    )

def analyze_business_highlights(ticker_symbol:str, year:str) -> str:
    businessHighlights = ReportAnalysisUtils.analyze_business_highlights(ticker_symbol, year)
    summarized = summarize(businessHighlights)
    return (
        f"##### Business Highlights\n"
        f"**Company Name:** {ticker_symbol}\n"
        f"**Business Highlights:** {summarized}\n"
        #f"{formatting_instructions}"
    )

def get_competitors_analysis(ticker_symbol:str, year:str) -> str:
    compAnalysis = ReportAnalysisUtils.get_competitors_analysis(ticker_symbol, year)
    summarized = summarize(compAnalysis)
    return (
        f"##### Competitor Analysis\n"
        f"**Company Name:** {ticker_symbol}\n"
        f"**Competitor Analysis:** {summarized}\n"
        #f"{formatting_instructions}"
    )

def get_risk_assessment(ticker_symbol:str, year:str) -> str:
    riskAssess = ReportAnalysisUtils.get_risk_assessment(ticker_symbol, year)
    summarized = summarize(riskAssess)
    return (
        f"##### Risk Assessment\n"
        f"**Company Name:** {ticker_symbol}\n"
        f"**Risk Assessment Analysis:** {summarized}\n"
        #f"{formatting_instructions}"
    )

def analyze_segment_stmt(ticker_symbol:str, year:str) -> str:
    segmentStmt = ReportAnalysisUtils.analyze_segment_stmt(ticker_symbol, year)
    summarized = summarize(segmentStmt)
    return (
        f"##### Segment Statement\n"
        f"**Company Name:** {ticker_symbol}\n"
        f"**Segment Statement Analysis:** {summarized}\n"
        #f"{formatting_instructions}"
    )

def analyze_cash_flow(ticker_symbol:str, year:str) -> str:
    cashFlow = ReportAnalysisUtils.analyze_cash_flow(ticker_symbol, year)
    summarized = summarize(cashFlow)
    return (
        f"##### Cash Flow\n"
        f"**Company Name:** {ticker_symbol}\n"
        f"**Cash Flow Analysis:** {summarized}\n"
        #f"{formatting_instructions}"
    )

def analyze_balance_sheet(ticker_symbol:str, year:str) -> str:
    balanceSheet = ReportAnalysisUtils.analyze_balance_sheet(ticker_symbol, year)
    summarized = summarize(balanceSheet)
    return (
        f"##### Balance Sheet\n"
        f"**Company Name:** {ticker_symbol}\n"
        f"**Balance Sheet Analysis:** {summarized}\n"
        #f"{formatting_instructions}"
    )

def analyze_income_stmt(ticker_symbol:str, year:str) -> str:
    incomeStmt = ReportAnalysisUtils.analyze_income_stmt(ticker_symbol, year)
    summarized = summarize(incomeStmt)
    return (
        f"#####Income Statement\n"
        f"**Company Name:** {ticker_symbol}\n"
        f"**Income Statement Analysis:** {summarized}\n"
        #f"{formatting_instructions}"
    )

In [ ]:
competitors = ["GOOGL","AMZN","NVDA"]

print(analyze_business_highlights("MSFT", "2024"))

Sec Api initialized
FMP api key found successfully.
Sec Api initialized
FMP api key found successfully.
***********************************
Calling summarize
***********************************
***********************************
##### Business Highlights
**Company Name:** MSFT
**Business Highlights:** Microsoft's performance in fiscal year 2024 showed significant growth across its business lines, driven by cloud services and strategic acquisitions. 

1. **Productivity and Business Processes**: The segment saw a 12% revenue increase, primarily due to strong growth in Office 365 Commercial and LinkedIn services. This growth was fueled by increased seat subscriptions and enhanced business solutions that drove higher revenue per user.

2. **Intelligent Cloud**: Revenue rose by 20%, with Azure leading the charge with a 30% increase in cloud services revenue. The segment's success was attributed to robust demand for Azure's consumption-based services and hybrid solutions like Windows Server

In [5]:
get_sec_report = FunctionTool(fmpUtils.get_sec_report, description="Get SEC Report")
display_image = FunctionTool(IPythonUtils.display_image, description="Display Image")
check_text_length = FunctionTool(TextUtils.check_text_length, description="Check Text Length")
build_annual_report = FunctionTool(ReportLabUtils.build_annual_report, description="Build Annual Report")
report_analysis = FunctionTool(ReportAnalysisUtils, description="Report Analysis")
report_chart_utils = FunctionTool(ReportChartUtils, description="Report Chart Utils")


In [7]:
analyst_agent = AssistantAgent(
    name="Expert_Investor",
    system_message=""" Role: Expert Investor
            Department: Finance
            Primary Responsibility: Generation of Customized Financial Analysis Reports

            Role Description:
            As an Expert Investor within the finance domain, your expertise is harnessed to develop bespoke Financial Analysis Reports that cater to specific client requirements. This role demands a deep dive into financial statements and market data to unearth insights regarding a company's financial performance and stability. Engaging directly with clients to gather essential information and continuously refining the report with their feedback ensures the final product precisely meets their needs and expectations.

            Key Objectives:

            Analytical Precision: Employ meticulous analytical prowess to interpret financial data, identifying underlying trends and anomalies.
            Effective Communication: Simplify and effectively convey complex financial narratives, making them accessible and actionable to non-specialist audiences.
            Client Focus: Dynamically tailor reports in response to client feedback, ensuring the final analysis aligns with their strategic objectives.
            Adherence to Excellence: Maintain the highest standards of quality and integrity in report generation, following established benchmarks for analytical rigor.
            Performance Indicators:
            The efficacy of the Financial Analysis Report is measured by its utility in providing clear, actionable insights. This encompasses aiding corporate decision-making, pinpointing areas for operational enhancement, and offering a lucid evaluation of the company's financial health. Success is ultimately reflected in the report's contribution to informed investment decisions and strategic planning.

            Reply TERMINATE when everything is settled.""",
    model_client=model_client,
    #tools=[get_sec_report, display_image, check_text_length, build_annual_report, report_analysis, report_chart_utils]
    tools=[get_sec_report]
)

In [8]:
team = RoundRobinGroupChat([analyst_agent], max_turns=50)

In [9]:
company = "Microsoft"
fyear = "2023"

message = dedent(
    f"""
    With the tools you've been provided, write an annual report based on {company}'s {fyear} 10-k report, format it into a pdf.
    Pay attention to the followings:
    - Explicitly explain your working plan before you kick off.
    - Use tools one by one for clarity, especially when asking for instructions. 
    - All your file operations should be done in "{work_dir}". 
    - Display any image in the chat once generated.
    - All the paragraphs should combine between 400 and 450 words, don't generate the pdf until this is explicitly fulfilled.
"""
)

stream = team.run_stream(task=message)
await Console(stream)

---------- user ----------

With the tools you've been provided, write an annual report based on Microsoft's 2023 10-k report, format it into a pdf.
Pay attention to the followings:
- Explicitly explain your working plan before you kick off.
- Use tools one by one for clarity, especially when asking for instructions. 
- All your file operations should be done in "report". 
- Display any image in the chat once generated.
- All the paragraphs should combine between 400 and 450 words, don't generate the pdf until this is explicitly fulfilled.

FMP api key found successfully.
---------- Expert_Investor ----------
[FunctionCall(id='call_H7eSCSY1YKb1vcUzPtDor3DZ', arguments='{"ticker_symbol":"MSFT","fyear":"latest"}', name='get_sec_report')]
---------- Expert_Investor ----------
[FunctionExecutionResult(content='Link: https://www.sec.gov/Archives/edgar/data/789019/000095017024087843/msft-20240630.htm\nFiling Date: 2024-07-30 00:00:00', call_id='call_H7eSCSY1YKb1vcUzPtDor3DZ')]
---------- Exp

TaskResult(messages=[TextMessage(source='user', models_usage=None, content='\nWith the tools you\'ve been provided, write an annual report based on Microsoft\'s 2023 10-k report, format it into a pdf.\nPay attention to the followings:\n- Explicitly explain your working plan before you kick off.\n- Use tools one by one for clarity, especially when asking for instructions. \n- All your file operations should be done in "report". \n- Display any image in the chat once generated.\n- All the paragraphs should combine between 400 and 450 words, don\'t generate the pdf until this is explicitly fulfilled.\n', type='TextMessage'), ToolCallRequestEvent(source='Expert_Investor', models_usage=RequestUsage(prompt_tokens=477, completion_tokens=264), content=[FunctionCall(id='call_H7eSCSY1YKb1vcUzPtDor3DZ', arguments='{"ticker_symbol":"MSFT","fyear":"latest"}', name='get_sec_report')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(source='Expert_Investor', models_usage=None, content=[FunctionE